<a href="https://colab.research.google.com/github/ashishar/data_science_codes/blob/main/King_county_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Visualation code credit:* https://www.kaggle.com/code/sauravvsrinku/starter-house-sales-in-king-county-8ceb15de-5 and https://www.kaggle.com/code/ronikdedhia/king-county-house-sales 

In [133]:
#import kaggle dataset
! pip install -q kaggle
from google.colab import files
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download harlfoxem/housesalesprediction



mkdir: cannot create directory ‘/root/.kaggle’: File exists
housesalesprediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
! unzip housesalesprediction.zip
!ls

Archive:  housesalesprediction.zip
replace kc_house_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# kc_house_data.csv has 21613 rows in reality, but we are only loading/previewing the first 1000 rows
data_houseing = pd.read_csv('kc_house_data.csv', delimiter=',', nrows = nRowsRead)
data_houseing.dataframeName = 'kc_house_data.csv'
nRow, nCol = data_houseing.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
data_houseing.head()

In [ ]:
data_houseing.describe().T

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
  
plotPerColumnDistribution(data_houseing, 10, 5)

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

plotCorrelationMatrix(data_houseing, 8)

In [ ]:
# Scatter plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()
plotScatterMatrix(data_houseing, 20, 10)

In [ ]:
sns.displot(x='price', data=data_houseing);

In [ ]:
sns.countplot(x='bedrooms', data=data_houseing);

In [ ]:
sns.scatterplot(x='price',y='sqft_living',data=data_houseing);

In [ ]:
# Map correlation with the cost of houses
plt.figure(figsize=(12,8))
sns.scatterplot(x='long',y='lat',data=data_houseing,hue='price');

In [ ]:
features = ['price','bedrooms','bathrooms','sqft_living','sqft_lot','floors','yr_built','grade']
sns.pairplot(data_houseing[features], size = 2.5)
plt.show()

In [ ]:
X = data_houseing.iloc[:,3:]
y = data_houseing.iloc[:,2:3]

In [ ]:
# X = data_houseing.loc[:, features]
# y = data_houseing.loc[:, ['price']]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .67)

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train,y_train)
y_pred = regr.predict(X_test)
#y_pred

In [ ]:
from sklearn.metrics import r2_score
print('Variance score: %.2f' % regr.score(X_test, y_test))
print("R^2 Score : %.2f" % r2_score(y_test,y_pred))

**Polynomial regression could give better results**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.fit_transform(X_test)
#X_train_poly

In [ ]:
regr1 = linear_model.LinearRegression()
regr1.fit(X_train_poly,y_train)
y_pred_poly=regr1.predict(X_test_poly)
#y_pred_poly

In [ ]:
print('Variance score: %.2f' % regr1.score(X_test_poly, y_test))
print("R^2 Score : %.2f" % r2_score(y_test,y_pred_poly))

In [ ]:
for i in range(1,5):
    poly = PolynomialFeatures(degree=i)
    x_train_poly = poly.fit_transform(X_train)
    x_test_poly = poly.fit_transform(X_test)
    regr1 = linear_model.LinearRegression()
    regr1.fit(X_train_poly,y_train)
    y_pred_poly=regr1.predict(X_test_poly)
    print(f"for degree = {i}")
    print('Variance score: %.2f' % regr1.score(X_test_poly, y_test))
    print("R^2 Score : %.2f" % r2_score(y_test,y_pred_poly))